# Imports

In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import findpeaks
from skimage import segmentation
from skimage import measure
import os
from prettytable import PrettyTable
from statistics import stdev
import sys

# Helper Functions

In [3]:
def Confusion(groundTruth, image):
    """
    Calculates true positive, true negative, false positive, false negative, false negative rate, sensitivity, 
    specificity, accuracy, false negative points, false negative points, jaccard, dice and precision recall curve
    Parameters:
    ----------------------
    groundTruth: Ground truth images
    image: image

    """
    x , y = groundTruth.shape
    T = np.count_nonzero(groundTruth)
    F = np.count_nonzero(groundTruth == 0)
    roc_point = []
    fn_points = []
    fp_points = []
    sensitivity = 0
    specificity = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(x):
        for j in range(y):
            if groundTruth[i,j] == 255 and image[i,j] == 255:
                tp += 1
            elif groundTruth[i,j] == 255 and image[i,j] == 0:
                fn += 1
                fn_points.append([i,j])
            elif groundTruth[i,j] == 0 and image[i,j] == 255:
                fp += 1
                fp_points.append([i,j])
            elif groundTruth[i,j] == 0 and image[i,j] == 0:
                tn += 1

    accuracy=(tp + tn)/(T+F)
    tp_rate = tp / T
    fp_rate = fp / F
    fn_rate = 0 if tp + fn == 0 else  fn / (tp + fn)
    dice = (2* tp)/((2*tp) + fp + fn) # Dice Coefficient
    jaccard = tp / (tp + fp + fn)  # Jaccard Index
    prc = 0 if tp + fp == 0 else  tp / (tp + fp)    # Precision-Recall 
    sensitivity= tp_rate # true positive rate
    specificity = tn / (tn + fp) # true negative rate
    roc_point=[fp_rate, tp_rate]
    
    return {
        "tp":tp/(T+F),
        "tn":tn/(T+F),
        "fp":fp/(T+F),
        "fn":fn/(T+F),
        "fn_rate":fn_rate,
        "sensitivity":sensitivity,
        "specificity":specificity,
        "accuracy":accuracy ,
        "roc_point":roc_point,
        "fn_points":fn_points,
        "fp_points":fp_points,
        "dice":dice,
        "jaccard":jaccard,
        "prc":prc,
    }

In [4]:
def histeq(img):
    """
    Histogram Equalization
    Parameters
    -----------------
    img: The gray image
    """
    return cv.equalizeHist(img)

In [5]:
def inverse(img):
    """
    Inverts the image
    Parameters
    -----------------
    img: The image to be inverted
    """
    return cv.bitwise_not(img)

In [6]:
def edgeEnhancement(img,t1 = 100 , t2 = 200,thresh = 100):
    """
    Applys edge enhancement on the image
    Parameters
    -----------------
    img: The image 
    t1: threshold 1
    t2: threshold 2
    thresh: threshold 
    """
    edges = cv.Canny(image=img, threshold1=t1, threshold2=t2) # Canny Edge Detection
    edges[edges == 255] = thresh
    imgEdg = np.subtract(img,edges)
    imgEdg[imgEdg < 0] = 0
    return imgEdg

In [7]:
def edgeEnhancement1(img):
    img = np.subtract(np.multiply(img,2),median(img,5))
    return img

In [8]:
def FR(img):
    """
    Apply Frost filter of image
    Parameters
    -----------------
    img: The image 
    """
    return findpeaks.frost_filter(img,3)

In [9]:
def median(img,size = 5):
    """
    Apply Median filter of image
    Parameters
    -----------------
    img: The image 
    """
    return cv.medianBlur(img,size)

In [10]:
def QS(img,ratio=0.8,kernel_size=4,max_dist=16):
    """
    Apply Quick Shift of an image
    Parameters
    -----------------
    img: The image 
    """
    imgSeg = segmentation.quickshift(img,ratio=ratio,kernel_size=kernel_size,max_dist=max_dist,convert2lab=False)
    
    ## coloriz regions with avg original color
    # save actual colors for every region 
    colorDic={}
    for indi,i in enumerate(imgSeg):
        for indj,j in enumerate(i):
            if (str(j) in colorDic):
                colorDic[str(j)].append(img[indi,indj])
            else:
                colorDic[str(j)] = [img[indi,indj]]
    # calculate avg color for every region 
    for i in colorDic:
        sum = 0
        count = len(colorDic[str(i)])
        for j in colorDic[str(i)]:
            sum+=j
        colorDic[str(i)] = sum / count
    # colorize every region with new color 
    for indi,i in enumerate(imgSeg):
        for indj,j in enumerate(i):
            imgSeg[indi,indj]=colorDic[str(j)]
    return imgSeg


In [11]:
def normalization(img,threshold = 0.8):
    """
    Apply normalization for image
    Parameters
    -----------------
    img: The image 
    threshold: Threshold
    """
    norm = cv.normalize(img, None, alpha=0, beta=1, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    norm[norm > threshold] = 255
    norm[norm <= threshold] = 0
    return norm

In [12]:
def clearBoarder(img):
    """
    Clears border of image
    Parameters
    -----------------
    img: The image 
    """
    return segmentation.clear_border(img)

In [13]:
def largestArea(img):
    """
    Gets the largest white area in the image
    Parameters
    -----------------
    img: The image 
    """
    img = img.copy()
    # get regions
    label_image = measure.label(img)
    imgreg=measure.regionprops(label_image.astype(int))
    maxAreaIndex = -1
    maxArea = -1
    # find region with max area
    for index,i in enumerate(imgreg):
        if(i.area > maxArea):
            maxAreaIndex= index
            maxArea= i.area
    # make all regions black except largest region
    for index,i in enumerate(imgreg):
        if(index == maxAreaIndex): continue
        cords = i.coords
        for c in cords:
            img[c[0],c[1]] = 0
    return img

In [14]:
def resultImg(img,gt):
    """
    Gets results parameters
    Parameters
    ---------------
    img: The image
    gt: Ground truth
    """
    results = Confusion(gt,img)
    imgresult = []
    for indi,i in enumerate(img):
        imgresult.append([])
        for j in i:
            if(j == 0):
                imgresult[indi].append([0, 0, 0]) 
            else:
                imgresult[indi].append([255, 255, 255])

    for i in results['fn_points']:
        imgresult[i[0]][i[1]] = [0, 0, 255]
    for i in results['fp_points']:
        imgresult[i[0]][i[1]] = [0, 255, 0]
    del results['fn_points']
    del results['fp_points']
    del results['roc_point']
    return imgresult,results

In [15]:
def printResults(results,path,names):
    """
    Prints the results in a table
    Parameters
    ---------------
    results: Output results
    path: Path to be saved in it
    """
    header = ["i"]
    data = []
    avg = [0] * (len(results[0])+1)
    std = []
    br=[]
    for i in range(len(results[0])+1):
        br.append('-----')
        std.append([])
    for i in results[0]:
        header.append(i)
    myTable = PrettyTable(header)
    for index,i in enumerate(results):
        data.append([])
        data[index].append(names[index])
        for j in i:
            num = [round(i[j][0],3),round(i[j][1],3)] if type(i[j]) is list else round(i[j],3)
            data[index].append(num)
        myTable.add_row(data[index])
    for i in data:
        for index,j in enumerate(i):
            if(type(j) is not str):
                avg[index]+=j
                std[index].append(j)
            else:
                std[index].append(0)
    for i in range(len(avg)):
        avg[i]=round(avg[i]/len(results),3)
        std[i]=round(stdev(std[i]),3)
    myTable.add_row(br)
    avg[0]='avg'
    myTable.add_row(avg)
    std[0]='std'
    myTable.add_row(std)
    print(myTable)
    original_stdout = sys.stdout
    with open(path+'results.txt', 'w') as f:   
        sys.stdout = f
        print(myTable)
        sys.stdout = original_stdout

In [16]:
def load_images_from_folder(folder):
    """
    Loads all images in the given folder
    Parameters
    -----------------
    folder: The path to the folder
    """
    images = []
    names = []
    for filename in os.listdir(folder):
        names.append(filename)
        img = cv.imread(os.path.join(folder,filename), 0)
        if img is not None:
            images.append(img)
    return images,names

# Get best threshold (training)

In [38]:
outPath = './results/QS-EDGE-MEDIAN-UDIAT/'
img_array = load_images_from_folder('data\Input')
gt_array = load_images_from_folder('data\GT')
isHistEq = {"12":True,"7":True,"2":True}
thresholdResults = {}
thresholds=[0.7,0.71,0.72,0.73,0.74,0.75,0.76,0.78,0.79,0.8,0.81,0.82,0.83,0.84,0.85,0.86,0.88,0.89,0.9]
for t in thresholds:
    thresholdResults[str(t)] = []
    for i in range(len(img_array)):
        img_array = load_images_from_folder('data\Input')
        img = img_array[i]
        
        gt = gt_array[i]
        
        if(str(i) in isHistEq):
            img = histeq(img)

        img = inverse(img)

        img = edgeEnhancement(img,100,130,150)

        img = median(img)
        
        img = QS(img)
        
        img = normalization(img,t)
        
        img = clearBoarder(img)
        
        img = largestArea(img)
        
        img,resultsData = resultImg(img,gt)
        thresholdResults[str(t)].append(resultsData['accuracy'])
for r in thresholdResults:
    sum=0
    for t in thresholdResults[r]:
        sum+=t
    thresholdResults[r]=sum/len(thresholdResults[r])

##### print best threshold

In [63]:
max = 0
thresh = 0
myTable = PrettyTable(['Threshold','Accuracy'])
for r in thresholdResults:
    myTable.add_row([r,round(thresholdResults[r],3)])
    if(thresholdResults[r]>max): 
        max=thresholdResults[r]
        thresh=r
myTable.add_row(['------------','-----------'])
myTable.add_row([thresh,max])
print(myTable)
original_stdout = sys.stdout
with open('./results/QS-EDGE-MEDIAN-UDIAT/threshold accuracy.txt', 'w') as f:   
    sys.stdout = f
    print(myTable)
    sys.stdout = original_stdout

+--------------+--------------------+
|  Threshold   |      Accuracy      |
+--------------+--------------------+
|     0.7      |       0.965        |
|     0.71     |       0.964        |
|     0.72     |       0.966        |
|     0.73     |       0.973        |
|     0.74     |       0.973        |
|     0.75     |       0.972        |
|     0.76     |       0.971        |
|     0.78     |        0.97        |
|     0.79     |       0.968        |
|     0.8      |       0.951        |
|     0.81     |       0.949        |
|     0.82     |       0.949        |
|     0.83     |       0.949        |
|     0.84     |       0.949        |
|     0.85     |       0.952        |
|     0.86     |       0.954        |
|     0.88     |       0.959        |
|     0.89     |       0.955        |
|     0.9      |       0.957        |
| ------------ |    -----------     |
|     0.73     | 0.9732952887146645 |
+--------------+--------------------+


# Main script

## Best Result 

In [17]:
# If the folder is not created it creates it
outPath = './results/QS-EDGE-MEDIAN-UDIAT/'
if not os.path.exists('./results/QS-EDGE-MEDIAN-UDIAT/'):
    os.makedirs('./results/QS-EDGE-MEDIAN-UDIAT/') 

# Loads input and ground truth images
img_array,img_names = load_images_from_folder('data\Input')
gt_array,gt_names = load_images_from_folder('data\GT')

# TODO: add images that need histogram equalization
isHistEq = {"000007.png","000019.png","000031.png"}
results = []
for i in range(len(img_array)):
    img = img_array[i]
    cv.imwrite(outPath+img_names[i] + '_0Input.jpg',img)
    gt = gt_array[i]
    if(img_names[i] in isHistEq):
        # Preprocessing 1: Hostogram Equalization (optional)
        img = histeq(img)
        cv.imwrite(outPath+img_names[i] + '_1Pre1.jpg',img)
    
    # Preprocessing 2: Inverting
    img = inverse(img)
    cv.imwrite(outPath+img_names[i] + '_1Pre2.jpg',img)

    # Preprocessing 3: Edge Enhancement
    img = edgeEnhancement(img,100,130,150)
    cv.imwrite(outPath+img_names[i] + '_1Pre3.jpg',img)
    
    # Preprocessing 4: Median Filter
    img = median(img)
    cv.imwrite(outPath + img_names[i] + '_1Pre4.jpg',img)
    
    # Segmentation: Quick Shift
    img = QS(img)
    cv.imwrite(outPath + img_names[i] + '_2Seg.jpg',img)

    # Postprocessing 1: Normalization
    img = normalization(img,0.73) # 0.73 is the best threshold regarding past section
    cv.imwrite(outPath + img_names[i] + '_3Post1.jpg',img)

    # Postprocessing 2: Clear Borders
    img = clearBoarder(img)
    cv.imwrite(outPath + img_names[i] + '_3Post2.jpg',img)
    
    # Postprocessing 3: Largest Area
    img = largestArea(img)
    cv.imwrite(outPath + img_names[i] + '_3Post3.jpg',img)
    
    # Output Image
    img,resultsData = resultImg(img,gt)
    cv.imwrite(outPath + img_names[i] + '_4Out.jpg',np.array(img))

    # Results Table
    results.append(resultsData)
    
printResults(results,outPath,img_names)

+------------+-------+-------+-------+-------+---------+-------------+-------------+----------+-------+---------+-------+
|     i      |   tp  |   tn  |   fp  |   fn  | fn_rate | sensitivity | specificity | accuracy |  dice | jaccard |  prc  |
+------------+-------+-------+-------+-------+---------+-------------+-------------+----------+-------+---------+-------+
| 000001.png | 0.007 | 0.991 |  0.0  | 0.002 |  0.187  |    0.813    |     1.0     |  0.998   | 0.896 |  0.811  | 0.997 |
| 000002.png | 0.036 | 0.959 |  0.0  | 0.004 |  0.096  |    0.904    |    0.999    |  0.996   | 0.944 |  0.893  | 0.987 |
| 000007.png | 0.026 | 0.941 | 0.014 | 0.019 |  0.419  |    0.581    |    0.985    |  0.967   | 0.614 |  0.443  | 0.651 |
| 000010.png | 0.005 | 0.993 | 0.002 | 0.001 |  0.157  |    0.843    |    0.998    |  0.997   | 0.781 |   0.64  | 0.727 |
| 000011.png | 0.028 | 0.946 | 0.001 | 0.025 |  0.472  |    0.528    |    0.999    |  0.974   | 0.679 |  0.514  | 0.953 |
| 000014.png | 0.076 |  